In [ ]:
from typing import Final

import matplotlib.pyplot as plt
import numpy as np

from qiskit import QuantumCircuit, execute
from qiskit.circuit import Qubit
from qiskit.providers import Backend
from qiskit.tools.monitor import job_monitor

In [ ]:
plt.rcParams["text.usetex"] = True
plt.rcParams["font.size"] = "12"

In [ ]:
NUM_SITES: Final[int] = 2**3

In [ ]:
def digital_sum(n) -> int:
    """Sum the digits of a number.

    Args:
        n: Number to sum the digits of

    Returns:
        int: Sum of digits in number
    """
    if len(n.split(" ")) == 2:
        n = n.split(" ")[0]
    return sum([int(i) for i in str(n)])

In [ ]:
# Bogoliubov transformation


def thetak(k: int, lambda_: float) -> float:
    """Calculate a theta_k given k and lambda.

    Args:
        k (int): k value
        lambda_ (float): lambda value

    Returns:
        float: value of theta
    """
    angle = 2 * np.pi * k / NUM_SITES
    x = np.cos(angle)
    y = np.sin(angle)

    return np.arccos((lambda_ - x) / np.sqrt((lambda_ - x) ** 2 + y**2))


def bogo(qc: QuantumCircuit, qubits: list[Qubit], theta: float) -> None:
    """Apply the Bogoliubov gate to a quantum circuit.

    Args:
        qc (QuantumCircuit): Qiskit Quantum Circuit. This circuit will be modified in place.
        qubits (list[Qubit]): List of Qubits to apply the circuits to. Often obtained with circuit.qubits
        theta (float): Angle to rotate in the X-axis by the controlled RX gate
    """
    qc.x(qubits[1])
    qc.cx(qubits[1], qubits[0])
    qc.crx(-theta, qubits[0], qubits[1])
    qc.cx(qubits[1], qubits[0])
    qc.x(qubits[1])

In [ ]:
def disentangle(qc: QuantumCircuit, lambda_: float) -> None:
    """Apply the disentaglement gates to the quantum circuit.

    Args:
        qc (QuantumCircuit): Qiskit quantum circuit. This circuit will be modified in place.
        lambda_ (float): lambda value
    """
    from lib import FFFT

    # bogo(qc, qc.qubits[0:2], thetak(0, lambda_))
    bogo(qc, qc.qubits[2:4], thetak(2, lambda_))
    bogo(qc, qc.qubits[4:6], thetak(1, lambda_))
    bogo(qc, qc.qubits[6:8], thetak(3, lambda_))

    qc.compose(FFFT(NUM_SITES), inplace=True)

In [ ]:
def expected_magnetization(
    lambda_: float, backend: Backend, shots: int = 1024
) -> float:
    """Execute the passed in circuit as a job and return the expected magnetization.

    Args:
        lambda_ (float): lambda value
        backend (Backend): backend to run the job against
        shots (int): number of shots to run the job [Default: 1024]

    Return:
        float: magnetization result from job
    """
    qc = QuantumCircuit(NUM_SITES)

    # Set correct ground state for lambda < 1
    if lambda_ < 1:
        qc.x(NUM_SITES - 1)

    disentangle(qc, lambda_)
    qc.measure_all()

    job = execute(qc, backend=backend, shots=shots)
    job_monitor(job)

    counts = job.result().get_counts()

    c1 = list(counts.keys())
    c2 = list(counts.values())

    return sum(
        [
            (1 - digital_sum(c1[j]) / (NUM_SITES / 2)) * c2[j] / shots
            for j in range(len(c1))
        ]
    )

In [ ]:
from qiskit_ibm_provider import IBMProvider

provider = IBMProvider()
provider.backends()

In [ ]:
backend = provider.get_backend("ibmq_qasm_simulator")

In [ ]:
def exact(lambda_: float) -> float:
    magnetization = lambda_ / (2 * np.sqrt(1 + lambda_**2))
    if lambda_ < 1:
        return magnetization
    if lambda_ > 1:
        return magnetization + 0.5
    return np.nan


vexact = np.vectorize(exact)

lambdas = np.linspace(0, 1.8, 11)
more_lambdas = np.linspace(0, 1.8, 301)

magnetization = [expected_magnetization(l, backend) for l in lambdas]

In [ ]:
# Plot the magnetization values for the range of lambdas
fix, ax = plt.subplots()

ax.plot(more_lambdas, vexact(more_lambdas), color="black", label="exact")
ax.plot(lambdas, magnetization, marker="o", label=backend.name)

ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"$\langle \sigma_{z} \rangle$")

ax.legend()

In [ ]:
def time_evolution(
    time: float, lambda_: float, backend: Backend, shots: int = 1024
) -> float:
    """Perform time evolution of the Ising Model.

    Args:
        time (float): time value to evolve over
        lambda_ (float): lambda value
        backend (Backend): name of the backend to run on
        shots (int): Number of shots to run the job [Default: 1024]

    Return:
        float: magnetization result
    """
    qc = QuantumCircuit(NUM_SITES)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        0,
    )
    qc.cx(0, 1)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        2,
    )
    qc.cx(2, 3)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        4,
    )
    qc.cx(4, 5)

    disentangle(qc, lambda_)
    qc.measure_all()

    qc.draw(output="mpl")

    job = execute(qc, backend=backend, shots=shots)
    job_monitor(job)

    counts = job.result().get_counts()

    c1 = list(counts.keys())
    c2 = list(counts.values())

    return sum(
        [
            (1 - digital_sum(c1[j]) / (NUM_SITES / 2)) * c2[j] / shots
            for j in range(len(c1))
        ]
    )

In [ ]:
def exact_time(time: float, lambda_: float) -> float:
    return (1 + 2 * lambda_**2 + np.cos(4 * time * np.sqrt(1 + lambda_**2))) / (
        2 + 2 * lambda_**2
    )


vexact_time = np.vectorize(exact_time)

times = np.linspace(0, 2, 9)
more_times = np.linspace(0, 2, 301)
lambdas = [0.5, 0.9, 1.8]

magnetization_time = [[time_evolution(t, l, backend) for t in times] for l in lambdas]

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap("tab10")

for idl, l in enumerate(lambdas):
    ax.plot(
        more_times,
        vexact_time(more_times, l),
        color=cmap(idl),
        label="exact" if idl == 0 else None,
    )
    ax.plot(
        times,
        magnetization_time[idl],
        linestyle="--",
        marker="o",
        label=r"$\lambda = " + str(l) + "$",
    )

ax.set_xlabel("time")
ax.set_ylabel(r"$\langle \sigma_{z} \rangle$")

ax.legend()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))

cmap = plt.get_cmap("tab10")

for idl, l in enumerate(lambdas):
    ax[idl].plot(
        more_times,
        vexact_time(more_times, l),
        color=cmap(idl),
    )
    ax[idl].plot(
        times, magnetization_time[idl], color=cmap(idl), linestyle="--", marker="o"
    )

    ax[idl].set_title(r"$\lambda = " + str(l) + "$")

    ax[idl].set_xlabel("time")
    ax[idl].set_ylabel(r"$\langle \sigma_{z} \rangle$")

In [ ]:
from qiskit import QuantumRegister
from lib.depolarizing import depolarizing_channel, calc_error_prob


def time_evolution_with_depolarizing_channel(
    time: float, lambda_: float, gamma: float, backend: Backend, shots: int = 128
) -> float:
    """Perform time evolution of the Ising Model. With a depolarizing channel attached to site 0.

    Args:
        time (float): time value to evolve over
        lambda_ (float): lambda value
        gamma (float): decay rate for depolarizing channel
        backend (Backend): name of the backend to run on
        shots (int): Number of shots to run the job [Default: 1024]

    Return:
        float: magnetization result
    """
    qIsing = QuantumRegister(NUM_SITES, "Ising")
    qDepolarizing = QuantumRegister(3, "Ancillae")
    qc = QuantumCircuit(qIsing, qDepolarizing)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        0,
    )
    qc.cx(0, 1)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        2,
    )
    qc.cx(2, 3)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        4,
    )
    qc.cx(4, 5)

    qc = depolarizing_channel(
        qc, calc_error_prob(gamma, time), qIsing[0], qDepolarizing # type: ignore
    )

    disentangle(qc, lambda_)
    qc.measure_all()

    qc.draw(output="mpl")

    job = execute(qc, backend=backend, shots=shots)
    job_monitor(job)

    counts = job.result().get_counts()

    c1 = list(counts.keys())
    c2 = list(counts.values())

    return sum(
        [
            (1 - digital_sum(c1[j]) / (NUM_SITES / 2)) * c2[j] / shots
            for j in range(len(c1))
        ]
    )

In [ ]:
# Run the time evolution with the depolarizing channel
times = np.linspace(0, 10, 45)
more_times = np.linspace(0, 10, 301)
lambdas = [0.5, 0.9, 1.8]
gamma = 0.5

magnetization_time = [
    [time_evolution_with_depolarizing_channel(t, l, gamma, backend) for t in times]
    for l in lambdas
]

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap("tab10")

idl = 0
l = lambdas[idl]

ax.plot(
    more_times,
    vexact_time(more_times, l),
    color=cmap(idl),
    label="closed",
)
ax.plot(
    times,
    magnetization_time[idl],
    linestyle="--",
    marker="o",
    label=r"$\lambda = " + str(l) + "$",
)

ax.set_xlabel("time")
ax.set_ylabel(r"$\langle \sigma_{z} \rangle$")

ax.legend()

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap("tab10")

idl = 1
l = lambdas[idl]

ax.plot(
    more_times,
    vexact_time(more_times, l),
    color=cmap(idl),
    label="closed",
)
ax.plot(
    times,
    magnetization_time[idl],
    linestyle="--",
    marker="o",
    color=cmap(idl),
    label=r"$\lambda = " + str(l) + "$",
)

ax.set_xlabel("time")
ax.set_ylabel(r"$\langle \sigma_{z} \rangle$")

ax.legend()

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap("tab10")

idl = 2
l = lambdas[idl]

ax.plot(
    more_times,
    vexact_time(more_times, l),
    color=cmap(idl),
    label="closed",
)
ax.plot(
    times,
    magnetization_time[idl],
    linestyle="--",
    marker="o",
    color=cmap(idl),
    label=r"$\lambda = " + str(l) + "$",
)

ax.set_xlabel("time")
ax.set_ylabel(r"$\langle \sigma_{z} \rangle$")

ax.legend()

In [ ]:
from lib.amplitudeDamping import amplitude_damping_channel_witness
from qiskit import QuantumRegister, ClassicalRegister


def time_evolution_with_amplitude_damping(
    time: float,
    lambda_: float,
    backend: Backend,
    observable: str,
    coupling: float,
    shots: int = 1024,
) -> float:
    """Perform time evolution of the Ising Model.

    Args:
        time (float): time value to evolve over
        lambda_ (float): lambda value
        backend (Backend): name of the backend to run on
        observable (str): measurement of the corresponding observable
        coupling (float): the coupling factor R
        shots (int): Number of shots to run the job [Default: 1024]

    Return:
        float: magnetization result
    """

    qr = QuantumRegister(NUM_SITES + 2)
    cr = ClassicalRegister(NUM_SITES)
    qc = QuantumCircuit(qr, cr)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        0,
    )
    qc.cx(0, 1)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        2,
    )
    qc.cx(2, 3)

    qc.u(
        np.arccos(lambda_ / np.sqrt(1 + lambda_**2)),
        (3 * np.pi / 2) + 4 * time * np.sqrt(1 + lambda_**2),
        0,
        4,
    )
    qc.cx(4, 5)

    disentangle(qc, lambda_)

    qc = amplitude_damping_channel_witness(qc, 0, 8, 9, observable, coupling, time)
    qc.measure_all()

    job = execute(qc, backend=backend, shots=shots)
    job_monitor(job)

    counts = job.result().get_counts()

    c1 = list(counts.keys())
    c2 = list(counts.values())

    return sum(
        [
            (1 - digital_sum(c1[j]) / (NUM_SITES / 2)) * c2[j] / shots
            for j in range(len(c1))
        ]
    )

In [ ]:
times = np.linspace(0, 2, 25)
more_times = np.linspace(0, 2, 301)
lambdas = [0.5, 0.9, 1.8]

magnetization_time_with_damping = [
    [time_evolution_with_amplitude_damping(t, l, backend, "xx", 0.2) for t in times]
    for l in lambdas
]
magnetization_time_with_damping_low = [
    [time_evolution_with_amplitude_damping(t, l, backend, "xx", 50) for t in times]
    for l in lambdas
]
magnetization_time_with_damping_med = [
    [time_evolution_with_amplitude_damping(t, l, backend, "xx", 100) for t in times]
    for l in lambdas
]
magnetization_time_with_damping_high = [
    [
        time_evolution_with_amplitude_damping(
            t,
            l,
            backend,
            "xx",
            400,
        )
        for t in times
    ]
    for l in lambdas
]

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap("tab10")

for idl, l in enumerate(lambdas):
    ax.plot(
        more_times,
        vexact_time(more_times, l),
        color=cmap(idl),
        label="exact" if idl == 0 else None,
    )
    ax.plot(
        times,
        magnetization_time_with_damping[idl],
        linestyle="--",
        marker="o",
        label=r"$\lambda = " + str(l) + "$",
    )

ax.set_xlabel("time")
ax.set_ylabel(r"$\langle \sigma_{z} \rangle$")

ax.legend()

In [ ]:
import qiskit.tools.jupyter

%qiskit_version_table